# Pipeline Example for SNLI

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchtext import data
from torchtext import datasets
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
from catalyst import dl
import wandb

DEVICE = 'cuda'

/home/denis.litvinov/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
I0228 17:11:40.498617 140194564380480 file_utils.py:41] PyTorch version 1.4.0 available.
I0228 17:11:41.141555 140194564380480 file_utils.py:57] TensorFlow version 2.0.0 available.


In [2]:
# create directory for checkpoints
logdir = 'logdir'
!rm -rf {logdir}
# !mkdir {logdir}

In [3]:
# model description in the notebook
# it's a good idea to parametrize network by fewer parameters, e.g. through h_size

class BiLstm(nn.Module):
    def __init__(self, vocab_size, h_size, n_layers, dropout, padding_idx):
        super(BiLstm, self).__init__()
        self.embedding = nn.Embedding(vocab_size, h_size, padding_idx=padding_idx, scale_grad_by_freq=True)
        self.lstm = nn.LSTM(h_size, h_size, n_layers, bidirectional=True, dropout=dropout)
        self.out = nn.Sequential(
            # here is the typical order of layers in FFNN
            nn.Dropout(dropout),
            nn.Linear(2*4*n_layers*h_size, h_size*3), # 2 sentences * 2 directions * lstm spec * n_layers * h_size
            nn.BatchNorm1d(h_size*3),
            nn.PReLU(),
            nn.Dropout(dropout),
            nn.Linear(h_size*3, 3)
        )
        
        self.init_weights()

    def forward(self, batch):
        prem, hyp = batch
        prem, prem_len = prem
        hyp, hyp_len = hyp
        
        prem, hyp = self.embedding(prem), self.embedding(hyp)
        
        # optimize perfomance of RNN by omitting padding token
        prem = nn.utils.rnn.pack_padded_sequence(prem, prem_len, enforce_sorted=False)
        hyp = nn.utils.rnn.pack_padded_sequence(hyp, hyp_len, enforce_sorted=False)
        _, h_prem = self.lstm(prem)
        _, h_hyp = self.lstm(hyp)
        
        h_prem = torch.cat(h_prem, dim=-1)
        h_prem = h_prem.permute(1,0,2)
        h_prem = h_prem.contiguous().view(h_prem.size(0),-1)
        
        h_hyp = torch.cat(h_hyp, dim=-1)
        h_hyp = h_hyp.permute(1,0,2)
        h_hyp = h_hyp.contiguous().view(h_hyp.size(0),-1)
        
        h = torch.cat([h_prem, h_hyp],dim=-1)
        return self.out(h)
    
    def init_weights(self):
        d = self.embedding.weight.size(1)
        nn.init.uniform_(self.embedding.weight, -1/np.sqrt(d), 1/np.sqrt(d))

In [4]:
%%time
# if a cell takes a long time to compute - use magic %%time to show others how much.

TEXT = data.Field(lower=True, tokenize='spacy', include_lengths=True) # batch_first = False by default in torchtext and pytorch
LABEL = data.LabelField(sequential=False, is_target=True)

train, valid, test = datasets.SNLI.splits(TEXT, LABEL)

CPU times: user 43.2 s, sys: 650 ms, total: 43.8 s
Wall time: 43.6 s


In [5]:
TEXT.build_vocab(train, valid, min_freq=5) # use min_freq to cutoff rare tokens
LABEL.build_vocab(train)

# show others important params of the model, if they are not obvious from your code
len(TEXT.vocab) 

16184

In [6]:
# Simple wrapper to join torchtext and catalyst API

class IteratorWrapper(DataLoader):
    __initialized__ = False

    def __init__(self, iter: iter):
        self.batch_size = iter.batch_size
        self.num_workers = 1
        self.collate_fn = None
        self.pin_memory = False
        self.drop_last = False
        self.timeout = 0
        self.worker_init_fn = None
        self.sampler = iter
        self.batch_sampler = iter
        self.__initialized__ = True

    def __iter__(self):
        return map(lambda batch: {
                    'features': (batch.premise, batch.hypothesis),
                    'targets': batch.label,
                }, self.batch_sampler.__iter__())

    def __len__(self):
        return len(self.batch_sampler)

In [7]:
# example of your hyperparameters 
# keep them as global vars or in the 'config' dict
n_layers = 3
h_size = 256
embed_dim = 300
num_epochs = 10 
dropout = 0.1
batch_size = 512 # BatchNorm do not work properly with small batch sizes
vocab_size = len(TEXT.vocab)

train_iter, valid_iter, test_iter = data.BucketIterator.splits((train, valid, test), batch_size=batch_size, device=DEVICE)
train_iter = IteratorWrapper(train_iter)
valid_iter = IteratorWrapper(valid_iter)
test_iter = IteratorWrapper(test_iter)
 
loaders = {'train': train_iter, 'valid': valid_iter}

model = BiLstm(
    vocab_size=vocab_size,
    h_size=h_size,
    n_layers=n_layers, 
    dropout=dropout,
    padding_idx=TEXT.vocab.stoi[TEXT.pad_token]
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=2, factor=0.5)

In [8]:
# use SupervisedWandbRunner runner to send statistics to wandb
runner = dl.SupervisedWandbRunner(DEVICE)
runner.train(model, 
             loaders=loaders,
             num_epochs=num_epochs,
             logdir=logdir,
             criterion=nn.CrossEntropyLoss(),
             optimizer=optimizer, 
             scheduler=scheduler,  
             callbacks=[
                dl.callbacks.CheckpointCallback(2), # save 2 best models (by epoch) into logdir
                dl.callbacks.AccuracyCallback(), # also show accuracy metric
                dl.callbacks.EarlyStoppingCallback(3), # stop training, if valid loss does not improve last 3 epochs
             ],
             # send current hyperparam values to wandb
             monitoring_params={
                 'entity': 'denaas', # your wandb username
                 'project': 'text-augmentation', # project name
                 'name': 'snli-example', # name of the specific run
                 'group': 'examples',
                 'config': {
                     'model': 'bilstm',
                     'optimizer': str(optimizer),
                     'scheduler': 'plateau',
                     'early_stop': 3,
                     'vocab_size': vocab_size,
                     'h_size': h_size,
                     'n_layers': n_layers,
                     'dropout': dropout,
                     'batch_size': batch_size,
                     'embed_dim': embed_dim,
                 },
             },
#              check=True, # set if you want to check pipeline for correctness, without actual training
             verbose=True)

wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory


wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
I0228 17:12:28.800946 140194564380480 run_manager.py:924] system metrics and metadata threads started
I0228 17:12:28.802671 140194564380480 run_manager.py:933] checking resume status, waiting at most 10 seconds
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path /home/denis.litvinov/Documents/hse_practice/msaidov/logdir/wandb/ wasn't writable, using system temp directory
wan

1/10 * Epoch (train):  12% 134/1073 [00:14<01:32, 10.16it/s, accuracy01=63.477, loss=0.806]

I0228 17:12:45.583782 140190032566016 run_manager.py:688] file/dir modified: /tmp/wandb/run-20200228_141228-yibs0k59/wandb-metadata.json


1/10 * Epoch (train):  24% 256/1073 [00:26<01:19, 10.24it/s, accuracy01=64.062, loss=0.781]

...

I0228 17:27:04.372694 140190032566016 run_manager.py:688] file/dir modified: /tmp/wandb/run-20200228_141228-yibs0k59/wandb-events.jsonl


8/10 * Epoch (train): 100% 1073/1073 [01:51<00:00,  9.62it/s, accuracy01=85.352, loss=0.354]
8/10 * Epoch (valid): 100% 20/20 [00:00<00:00, 30.36it/s, accuracy01=72.807, loss=0.738]
Early stop at 7 epoch
[2020-02-28 17:27:08,977] 
8/10 * Epoch 8 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=19432.1934 | _timers/batch_time=0.0272 | _timers/data_time=0.0087 | _timers/model_time=0.0185 | accuracy01=86.5338 | loss=0.3510
8/10 * Epoch 8 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=27712.7028 | _timers/batch_time=0.0210 | _timers/data_time=0.0069 | _timers/model_time=0.0141 | accuracy01=77.7712 | loss=0.6380


I0228 17:27:08.977583 140194564380480 logging.py:153] 
8/10 * Epoch 8 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=19432.1934 | _timers/batch_time=0.0272 | _timers/data_time=0.0087 | _timers/model_time=0.0185 | accuracy01=86.5338 | loss=0.3510
8/10 * Epoch 8 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=27712.7028 | _timers/batch_time=0.0210 | _timers/data_time=0.0069 | _timers/model_time=0.0141 | accuracy01=77.7712 | loss=0.6380
I0228 17:27:08.980981 140194564380480 run_manager.py:1068] shutting down system stats and metadata service


Top best models:
logdir/checkpoints/train.5.pth	0.5776
logdir/checkpoints/train.4.pth	0.5795


I0228 17:27:09.378340 140190032566016 run_manager.py:688] file/dir modified: /tmp/wandb/run-20200228_141228-yibs0k59/wandb-history.jsonl
I0228 17:27:09.381037 140190032566016 run_manager.py:688] file/dir modified: /tmp/wandb/run-20200228_141228-yibs0k59/wandb-summary.json
I0228 17:27:09.384158 140190032566016 run_manager.py:688] file/dir modified: /tmp/wandb/run-20200228_141228-yibs0k59/wandb-events.jsonl
I0228 17:27:09.386618 140190032566016 run_manager.py:677] file/dir created: /tmp/wandb/run-20200228_141228-yibs0k59/log.txt
I0228 17:27:09.388877 140190032566016 run_manager.py:677] file/dir created: /tmp/wandb/run-20200228_141228-yibs0k59/train_log/events.out.tfevents.1582899151.UNIT-1482.10852.0
I0228 17:27:09.393844 140190032566016 run_manager.py:677] file/dir created: /tmp/wandb/run-20200228_141228-yibs0k59/valid_log/events.out.tfevents.1582899256.UNIT-1482.10852.1
I0228 17:27:09.397264 140190032566016 run_manager.py:677] file/dir created: /tmp/wandb/run-20200228_141228-yibs0k59/t

In [9]:
#  Evaluate on the test set

runner = dl.SupervisedRunner()
y_pred = runner.predict_loader(model, test_iter, verbose=True)
# Do not forget, that the NN compute the class logits, from which you actually have to make prediction
y_pred = np.argmax(y_pred, axis=1)
y_true = np.concatenate([x['targets'].cpu().numpy() for x in test_iter])

test_score = accuracy_score(y_true, y_pred)
print(test_score)

# send test score to wandb
wandb.log({'acc/test': test_score})

1/1 * Epoch (infer): 100% 20/20 [00:00<00:00, 31.74it/s]
0.7769747557003257


I0228 17:27:13.311534 140194564380480 run_manager.py:924] system metrics and metadata threads started
I0228 17:27:13.314374 140194564380480 run_manager.py:933] checking resume status, waiting at most 10 seconds
I0228 17:27:13.996902 140189225703168 run_manager.py:688] file/dir modified: /tmp/wandb/run-20200228_141228-yibs0k59/wandb-metadata.json
I0228 17:27:14.627063 140194564380480 run_manager.py:951] resuming run from id: UnVuOnYxOnlpYnMwazU5OnRleHQtYXVnbWVudGF0aW9uOmRlbmFhcw==
I0228 17:27:14.650969 140194564380480 run_manager.py:963] upserting run before process can begin, waiting at most 10 seconds
I0228 17:27:14.950582 140190032566016 run_manager.py:1048] saving patches
I0228 17:27:15.279410 140190032566016 run_manager.py:1052] saving pip packages
I0228 17:27:15.281081 140190032566016 run_manager.py:1054] initializing streaming files api
I0228 17:27:15.281960 140190032566016 run_manager.py:1061] unblocking file change observer, beginning sync with W&B servers
I0228 17:27:15.283294